In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.datasets.data_utils import get_file
from keras.callbacks import Callback

import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell

import numpy as np
import random
import sys
import csv
import matplotlib.pyplot as plt

synthetic_data_set = "../syntheticData/naive_c5_q50_s4000_v0.csv"

Using Theano backend.
Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
# Parameters
# learning_rate = 0.001
# training_iters = 10
# num_hidden = 512
train_split = 0.7
#batch_size = 128
#display_step = 1

In [3]:
# Read in the data set
data_array = np.array(list(csv.reader(open(synthetic_data_set,"rb"),delimiter=','))).astype('int')
num_samples = data_array.shape[0]
num_problems = data_array.shape[1]

# Split data into train and test (70/30 split)
train = data_array[0:int(num_samples*train_split),:]
test = data_array[int(num_samples*train_split):num_samples,:]

num_train = train.shape[0]
num_test = test.shape[0]

In [4]:
# Create 3-dimensional input tensor with one-hot encodings for each sample
# the dimension of each vector for a student i and time t is 2 * num_problems
# where the first half corresponds to the correctly answered problems and the
# second half to the incorrectly answered ones.

# The output tensor is also 3-dimensional where the output dimension is 
# num_problems and is encoded as 1 if the problem is correct and -1 if it is incorrect.
# Since the output is only used in the loss computation, as long as we 
# configure the loss objective correctly, we will not lose any specificity
# in not using one-hot encoding.

print('Vectorization...')
X_train = np.zeros((num_train, num_problems, num_problems * 2), dtype=np.bool)
y_train = np.zeros((num_train, num_problems, num_problems), dtype=np.bool)

for i in xrange(num_train):
    for t in xrange(num_problems):
        if train[i,t] == 1:
            X_train[i, t, t] = 1
            y_train[i, t, t] = 1
        else:
            X_train[i, t, num_problems + t] = 1
            y_train[i, t, t] = -1
            
#     y_train[i,0:num_problems - 1] = train[i,1:num_problems]
#     y_train[i, num_problems-1] = 0
    
print('Finished vectorization...')

Vectorization...
Finished vectorization...


In [5]:
# The objective function should be placed in keras codebase for objectives
def my_objective(y_true, y_pred):
    return K.binary_crossentropy(K.dot(y_pred, y_true), K.sum(y_true)*0.5+0.5)

# Print for verification
print(X_train[10])
print(y_train[10])


[[False False False ..., False False False]
 [False  True False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False  True]]
[[ True False False ..., False False False]
 [False  True False ..., False False False]
 [False False  True ..., False False False]
 ..., 
 [False False False ...,  True False False]
 [False False False ..., False  True False]
 [False False False ..., False False  True]]


In [8]:
# build the model: 2 stacked LSTM
print('Building model...')
model = Sequential()
model.add(LSTM(num_problems, return_sequences=True, input_shape=(num_problems, num_problems * 2)))
# model.add(LSTM(num_problems, return_sequences=True, input_shape=(num_problems, num_problems * 2)))
# model.add(Dropout(0.2))
# model.add(LSTM(512, return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(num_problems))
model.add(Activation('softmax'))

print('Compile model...')
model.compile(loss="my_objective", optimizer='rmsprop', class_mode='binary')

print ('Compiling finished.')

Building model...
Compile model...
Compiling finished.


In [9]:
model.summary()
x = model.to_json()

--------------------------------------------------------------------------------
Initial input shape: (None, 50, 100)
--------------------------------------------------------------------------------
Layer (name)                  Output Shape                  Param #             
--------------------------------------------------------------------------------
LSTM (Unnamed)                (None, 50, 50)                30200               
Activation (Unnamed)          (None, 50, 50)                0                   
--------------------------------------------------------------------------------
Total params: 30200
--------------------------------------------------------------------------------


In [10]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

# train the model, output generated text after each iteration
numEpochs = 5
histories = [LossHistory() for i in xrange(numEpochs)]

for i in xrange(numEpochs):
    print()
    print('-' * 50)
    print('Iteration', i)
    model.fit(X_train, y_train, batch_size=200, nb_epoch=1, verbose=1, show_accuracy=True, callbacks=[histories[i]])
    


--------------------------------------------------
Iteration 0
Epoch 1/1
2800/2800 [==============================] - 117s - loss: 19465.4646 - acc: 0.9800   

--------------------------------------------------
Iteration 1
Epoch 1/1
2800/2800 [==============================] - 116s - loss: 19463.8083 - acc: 0.9800   

--------------------------------------------------
Iteration 2
Epoch 1/1
2800/2800 [==============================] - 124s - loss: 19463.0749 - acc: 0.9800   

--------------------------------------------------
Iteration 3
Epoch 1/1
1200/2800 [===========>..................] - ETA: 73s - loss: 19462.6895 - acc: 0.9800

KeyboardInterrupt: 

In [11]:
losses = []
for h in histories:
    losses += h.losses
losses

AttributeError: 'LossHistory' object has no attribute 'losses'

In [12]:
plt.plot(xrange(len(losses)), losses)

# plot the trend line with error bars that correspond to standard deviation
plt.title('Loss History')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.show()
